In [ ]:
import nltk
import os
import pandas as pd
import pyspark.sql.functions as F
import string
from pyspark import sql
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.sql.types import *
from pyspark.ml.feature import Word2Vec, Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel, NaiveBayes, NaiveBayesModel, LinearSVC, LinearSVCModel
from pyspark.ml.clustering import LDA, LDAModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import matplotlib.pyplot as plt
import numpy as np


In [2]:
sparkConf = SparkConf().setAppName('CMPE256')
sparkConf.set('spark.executor.memory', '16g')
sparkConf.set('spark.executor.cores', '5')
sparkConf.set('spark.cores.max', '40')
sparkConf.set('spark.driver.memory', '12g')
sparkConf.set('spark.driver.maxResultSize', '4g')
DEBUG_CHECKPOINT_PATH = 'C:/Users/mayan/Desktop/college/256'
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
spark.sparkContext.setCheckpointDir(DEBUG_CHECKPOINT_PATH)


In [3]:
df=spark.read.csv("cleaned_hm.csv")

In [ ]:
df

In [4]:
# df_hm = df[df['cleaned_hm'].notnull()]
gender_train=spark.read.csv("train.csv",header=True)
gender_test=spark.read.csv("test.csv",header=True)


In [5]:
gender_train.take(5)

[Row(data='i went on a successful date with someone i felt sympathy and connection with', gender_bin='1'),
 Row(data='i played a new game that was fun and got to enjoy the mechanics of it', gender_bin='1'),
 Row(data='i listened to some music and heard an entire album i hadn t heard more than once before', gender_bin='1'),
 Row(data='went to see a movie with my friend', gender_bin='1'),
 Row(data='played guitar learning a song on it', gender_bin='1')]

In [6]:
tokenizer = Tokenizer(inputCol="data", outputCol="tokens")
word2vec = Word2Vec(vectorSize=200, minCount=10, numPartitions=10, inputCol=tokenizer.getOutputCol(), outputCol="features")
pipeline = Pipeline(stages=[tokenizer, word2vec])
pipelineModel = pipeline.fit(gender_train)

In [ ]:
gende

In [7]:
pTrainDF = pipelineModel.transform(gender_train)
pTestDF = pipelineModel.transform(gender_test)

In [9]:
pTrainDF = pTrainDF.withColumn('gender_bin', pTrainDF['gender_bin'].cast(IntegerType()))
pTestDF = pTestDF.withColumn('gender_bin', pTestDF['gender_bin'].cast(IntegerType()))

In [10]:
pTestDF.take(5)

[Row(data='i started to walk more around the office meeting my step goal', gender_bin=0, tokens=['i', 'started', 'to', 'walk', 'more', 'around', 'the', 'office', 'meeting', 'my', 'step', 'goal'], features=DenseVector([0.0237, 0.0018, -0.019, -0.0228, -0.0141, 0.0338, -0.034, 0.0489, 0.0257, -0.0364, 0.0306, 0.0118, 0.0304, -0.0016, 0.0068, 0.0446, -0.0038, 0.0247, -0.0085, 0.0353, -0.0104, -0.0051, -0.019, 0.0116, 0.0063, 0.0189, -0.0844, 0.01, -0.012, -0.0526, 0.0171, 0.0332, -0.0068, -0.0495, 0.0263, 0.0004, -0.0022, 0.008, -0.0101, 0.0667, 0.0116, -0.0084, 0.0002, 0.0625, -0.0155, 0.0463, -0.0395, 0.0082, 0.0422, -0.0088, -0.0129, -0.0, 0.0045, 0.0001, -0.0004, 0.0071, -0.0062, -0.0185, -0.0262, -0.0109, -0.0321, 0.0032, 0.0006, -0.0127, -0.0202, -0.0008, 0.0183, 0.019, -0.0228, -0.0106, -0.0028, -0.0431, 0.0188, -0.0422, 0.0703, -0.0277, 0.0332, 0.024, 0.0324, 0.0098, 0.0041, -0.0149, 0.0769, 0.007, -0.0181, -0.0467, -0.0167, -0.058, -0.0187, -0.021, 0.0041, -0.0014, 0.0071, -0.008

In [11]:
rForest = RandomForestClassifier(labelCol='gender_bin', featuresCol='features')
rForestModel = rForest.fit(pTestDF)

In [12]:
predictions = rForestModel.transform(pTestDF)

In [13]:
evaluator = MulticlassClassificationEvaluator(labelCol="gender_bin", predictionCol="prediction", metricName="f1")
evaluator.evaluate(predictions)

0.5677860358470923

In [14]:
lr = LogisticRegression(featuresCol='features', labelCol='gender_bin')
lrModel = lr.fit(pTrainDF)
predictionsLR = lrModel.transform(pTestDF)
evaluator.evaluate(predictionsLR)

0.5334780715948739

In [16]:
# naiveBayes = NaiveBayes(featuresCol='features', labelCol='gender_bin')
# naiveModel = naiveBayes.fit(pTrainDF)
# predictionsNaive = naiveModel.transform(pTestDF)
# evaluator.evaluate(predictionsNaive)